In [22]:
#Libraries
import pandas as pd
import numpy as np
from etl.utils import read_sql_table
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import statsmodels.api as sm

In [3]:
df = read_sql_table("gold_cpw")

###Variable manipulation
# Convert HourDK to datetime if it's not already in datetime format
df['hour_utc'] = pd.to_datetime(df['hour_utc'])

# Extract the hour from the HourDK column
df['hour'] = df['hour_utc'].dt.hour

#Subsetting variables
df = df[['hour', 'consumption_kwh', 'spot_price_dkk', 'temp_mean_past1h', 'wind_speed_past1h',
    'humidity_past1h', 'precip_past1h']]
#Converting variables to float64
df['hour'] = df['hour'].astype('float64')
df['spot_price_dkk'] = df['spot_price_dkk'].astype('float64')

print(df.head().to_string(), "\n")
print(df.info())

   hour  consumption_kwh  spot_price_dkk  temp_mean_past1h  wind_speed_past1h  humidity_past1h  precip_past1h
0   0.0       143408.914      383.950012               5.2                3.7             88.0            0.0
1   1.0       136154.493      387.230011               4.5                3.9             91.0            0.0
2   2.0       130648.111      386.929993               3.8                4.0             93.0            0.0
3   3.0       126554.756      387.829987               3.2                4.2             95.0            0.0
4   4.0       131530.892      391.480011               3.3                4.1             96.0            0.0 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647 entries, 0 to 646
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   hour               647 non-null    float64
 1   consumption_kwh    647 non-null    float64
 2   spot_price_dkk     647 non-null    fl

In [4]:
##Checking for missing values
print(df.isna().any())
#No missing values

hour                 False
consumption_kwh      False
spot_price_dkk       False
temp_mean_past1h     False
wind_speed_past1h    False
humidity_past1h      False
precip_past1h        False
dtype: bool


In [5]:
##Splitting dataset
X = df[['hour', 'spot_price_dkk', 'temp_mean_past1h', 'wind_speed_past1h', 'humidity_past1h', 'precip_past1h']]
y = df[['consumption_kwh']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
print(X_train)

     hour  spot_price_dkk  temp_mean_past1h  wind_speed_past1h  \
171   3.0      494.239990              -1.0                1.1   
477  21.0      716.400024               4.6                1.4   
249   9.0      461.429993               2.4                6.2   
574  22.0      469.649994               4.1                2.0   
185  17.0      624.400024               0.1                1.7   
..    ...             ...               ...                ...   
98    2.0      420.089996               2.4                4.5   
322  10.0      371.649994               8.7                5.2   
382  22.0      422.209991               1.5                3.3   
365   5.0      352.410004               5.7                4.4   
510   6.0      477.450012               6.9                4.2   

     humidity_past1h  precip_past1h  
171             97.0            0.0  
477             99.0            0.0  
249             83.0            0.0  
574             97.0            0.1  
185             9

In [6]:
####Linear Regression####
##Training and evaluating linear regression model without preprocessing
est = sm.OLS(y_train, sm.add_constant(X_train)) #model with constant
est_fit = est.fit()
print(est_fit.summary()) #All but hour and precip_past1h are significant
#In a zero-intercept model, all but precip_past1h and spot_price_dkk are significant


                            OLS Regression Results                            
Dep. Variable:        consumption_kwh   R-squared:                       0.139
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     13.76
Date:                Mon, 06 May 2024   Prob (F-statistic):           1.64e-14
Time:                        17:52:27   Log-Likelihood:                -6404.1
No. Observations:                 517   AIC:                         1.282e+04
Df Residuals:                     510   BIC:                         1.285e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               2.65e+05   2.98e+0

In [7]:
#Linear regression with scaling
X_train_scale = StandardScaler().fit_transform(X_train)
est = sm.OLS(y_train, sm.add_constant(X_train_scale))
est_fit = est.fit()
print(est_fit.summary())
#Conclude on variable importance based on coefficients...

                            OLS Regression Results                            
Dep. Variable:        consumption_kwh   R-squared:                       0.139
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     13.76
Date:                Mon, 06 May 2024   Prob (F-statistic):           1.64e-14
Time:                        17:52:29   Log-Likelihood:                -6404.1
No. Observations:                 517   AIC:                         1.282e+04
Df Residuals:                     510   BIC:                         1.285e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.006e+05   2567.816     78.117      0.0

In [23]:
###Prediction accuracy - linear regression
X_test_scale = StandardScaler().fit_transform(X_test)
y_pred_lm = est_fit.predict(sm.add_constant(X_test_scale))
rmse_lm = root_mean_squared_error(y_test, y_pred_lm)
print(round(rmse_lm)) #3,498,841,479

59151


In [24]:
###Random Forest
#Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
rf_model = RandomForestRegressor(n_estimators = 200, bootstrap=True) #200 trees
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test) #Test
rmse_rf = root_mean_squared_error(y_test, y_pred_rf)
print(round(rmse_rf)) #1,540,289,685
#Performs significantly better than lm

/Users/frederikjacobsen/Repos/au-data-science-project/.venv/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


39389


In [25]:
###Support Vector Machines
#Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html
svm_model = SVR(C=1)
svm_model.fit(X_train, y_train) #Train
y_pred_svm = svm_model.predict(X_test) #Test
rmse_svm = root_mean_squared_error(y_test, y_pred_svm) #RMSE
print(round(rmse_svm)) #3,906,756,583

62504


/Users/frederikjacobsen/Repos/au-data-science-project/.venv/lib/python3.9/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
#Comparing models
rmse_data = {'Model': ['Linear Model', 'Random Forrest', 'Support Vector Machine'],
            'RMSE': [rmse_lm, rmse_rf, rmse_svm]
    }

rmse_df = pd.DataFrame(rmse_data) #Make dataframe of RMSE data

print(rmse_df)

                    Model           MSE
0            Linear Model  3.498841e+09
1          Random Forrest  1.628611e+09
2  Support Vector Machine  3.906757e+09


Implementing cross validation on the three models

In [11]:
#Linear model CV
# Define a pipeline with StandardScaler and LinearRegression
pipeline = make_pipeline(StandardScaler(), LinearRegression())

# Perform cross-validation
rmse_scores_lm = -cross_val_score(pipeline, X_train, y_train, cv=10, scoring='neg_mean_squared_error')

# Calculate mean and standard deviation of RMSE scores
mean_rmse_lm = np.mean(rmse_scores_lm)
std_rmse_lm = np.std(rmse_scores_lm)

# Print mean squared error from cross-validation
print("Root Mean Squared Error (Cross-validation):", round(mean_rmse_lm))
print("Standard Deviation of RMSE (Cross-validation):", round(std_rmse_lm))

Mean Squared Error (Cross-validation): 3531881852
Standard Deviation of MSE (Cross-validation): 1527924608


In [26]:
#Random forest CV
rf_model = RandomForestRegressor(n_estimators=200, bootstrap=True)

#Cross-validation
rmse_scores_rf = -cross_val_score(rf_model, X, y, cv=10, scoring='neg_root_mean_squared_error')

# Calculate mean and standard deviation of RMSE scores
mean_rmse_rf = np.mean(rmse_scores_rf)
std_rmse_rf = np.std(rmse_scores_rf)

# Train model on full training data
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)

# Calculate rmse on the test set
rmse_rf = root_mean_squared_error(y_test, y_pred_rf)

# Print rmse from cv and on test set
print("Root Mean Squared Error (Cross-validation):", round(mean_rmse_rf)) #2,991,619,458
print("Standard Deviation of RMSE (Cross-validation):", round(std_rmse_rf)) #Large variation in rmse scores.
#Suggests that the splits from the small dataset has large influence on performance.
print("Root Mean Squared Error (Test set):", round(rmse_rf))

/Users/frederikjacobsen/Repos/au-data-science-project/.venv/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/frederikjacobsen/Repos/au-data-science-project/.venv/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/frederikjacobsen/Repos/au-data-science-project/.venv/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/frederikjacobsen/Repos/au-data-science-project/.venv/l

Mean Squared Error (Cross-validation): 50940
Standard Deviation of RMSE (Cross-validation): 19242
Mean Squared Error (Test set): 39410


In [27]:
#Support Vector Machines CV
svm_model = SVR(C=100000)

#CV
rmse_scores_svm = -cross_val_score(svm_model, X, y, cv=10, scoring='neg_root_mean_squared_error')

# Calculate mean and standard deviation of RMSE scores
mean_rmse_svm = np.mean(rmse_scores_svm)
std_rmse_svm = np.std(rmse_scores_svm)

print("Root Mean Squared Error (Cross-validation):", round(mean_rmse_svm))
print("Standard Deviation of RMSE (Cross-validation):", round(std_rmse_svm))

Root Mean Squared Error (Cross-validation): 55373
Standard Deviation of RMSE (Cross-validation): 24705


/Users/frederikjacobsen/Repos/au-data-science-project/.venv/lib/python3.9/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/frederikjacobsen/Repos/au-data-science-project/.venv/lib/python3.9/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/frederikjacobsen/Repos/au-data-science-project/.venv/lib/python3.9/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/frederikjacobsen/Repos/au-data-science-project/.venv/lib/pyt